In [56]:
import pandas as pd
import numpy as np

In [57]:
import re, collections

In [58]:
from os.path import dirname, join

In [59]:
def clean_up (file_name):
    with open(file_name) as infile:
        contents = infile.read()
        # replacements
        rep = {'%20': ' ','%2F%2F':'://', '%2F':'/',';':'\n','.png':'.png \n','%3A':''}
        for i, j in rep.iteritems():
            contents = contents.replace(i, j)
    return contents
        

In [60]:
data = clean_up('./Test_results/alexa200_data.txt')

In [61]:
def prepare(file_name, from_file=False):
    collection = []
    data = collections.defaultdict(dict)
    count_num_data = 0
    
    # checking for expernal or internal file source
    if from_file:
        with open(file_name, 'r') as l_file:
            content = l_file.read()
    else:
        content = file_name
    
    # separate each param and add to list
    line = content.split('\n')
    
    for each in line:
        
        values = each.split(':')
        
        # forming params and value pairs
        if len(values) == 2:
            key, value = values
        elif len(values) == 3:
            key, value_1, value_2 = values
            value = value_1+":"+value_2
            if key == 'web address':
                catagory = value
        else:
            pass
        
        # put the value for each param of single website together
        data[catagory].setdefault(key,[]).append(value)
        #if key not in ['web address', 'Waterfall view']:
           # print data[catagory].setdefault(key)
      
        cleaned_dict = dict(data)
      

    for web in cleaned_dict.keys():
        count_num_data = count_num_data + 1
        #print count_num_data
        #print web
        del cleaned_dict[web]['Waterfall view']
        del cleaned_dict[web]['web address']

        for x in cleaned_dict[web].keys():
            if '' in cleaned_dict[web][x]:
                for y in cleaned_dict[web][x]:
                    cleaned_dict[web][x] = np.nan
        if cleaned_dict[web].has_key(''):
             del cleaned_dict[web]['']

        else:
            pass
    #print "The number of data collected is: ", count_nam_data

    return cleaned_dict

In [62]:
data=prepare(data)

In [63]:
def select(dataframe, param):
    dfT = dataframe.transpose()
    
    # covert back to frame
    df = pd.DataFrame(data=dfT[param]).dropna()
    
    # this link can be changed to .mean() for avarge
    # .min() for the minmun value
    # .max() for the max value
    s = df.apply(lambda x: pd.Series(x[param]), axis = 1).transpose().max()
    s.name = param
    
    # conver serias to DataFrame
    df_return = df.drop(param, axis=1).join(s)
    
    return df_return
    

In [64]:
df = pd.DataFrame(data=data)

In [65]:

def table(df):
    params = ['(Doc complete) Byets in', '(Doc complete) Requests','(Fully loaded) Bytes in','(Fully loaded) Requests',
          '(Fully loaded) Time','DOM elements','First byte','Load time','Speed Index','Start render',]
 
    df_list = [select(df, param) for param in params]
    
    for indx, i in enumerate(df_list):
        
        # first time run
        if indx == 0:
            joined = i
        
        if indx < len(df_list) -1:
        
            joined = joined.join(df_list[indx+1])
            
    
       
    joined.index.names = ['Site name']

    return joined
#     df1 = pd.DataFrame(data=df_list[:1])
#     print df1
#     df2 = pd.DataFrame(data=df_list[1:2])
    #return pd.merge(*df_list, how='outer')
    

   

In [66]:
dfnew = table(df)
#dfnew

In [67]:
# dfnew.to_csv('./bokeh_app/data/formatetop200.csv')

## we can start making the model now

## Algorithm:

### Step1: Find min and max of each of the 7 parameters.

In [68]:
df_set = pd.read_csv('./bokeh_app/data/formatetop100.csv')

params = ['(Doc complete) Byets in', '(Doc complete) Requests','(Fully loaded) Bytes in','(Fully loaded) Requests',
          '(Fully loaded) Time','DOM elements','First byte','Load time','Speed Index','Start render',]
# list 
param_values = {}
for param in params:
    param_values.update({
    param:[df_set[param].min(), df_set[param].mean(), df_set[param].max()],})


# for key in param_values:
#     print param_values[key]

df_param = pd.DataFrame.from_dict(param_values, orient='index')
df_param.columns=['Min','Average','Max']
#df_param
#df_param = pd.DataFrame(list(param_values.items()),columns=['Min','Average','Max'])


In [69]:
df_set.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
(Doc complete) Byets in,49,2283267.530612,2988632.278208,2999,527020,1159728,2853885,13786583
(Doc complete) Requests,49,79.020408,73.258529,5,24,61,85,369
(Fully loaded) Bytes in,49,2725607.244898,3601853.629268,3776,609950,1389638,3228037,17350231
(Fully loaded) Requests,49,109.795918,101.120180,6,39,71,130,408
(Fully loaded) Time,49,9204.122449,9720.931734,724,3191,4692,11162,43134
DOM elements,49,1123.020408,1113.947532,25,399,677,1421,6261
First byte,49,586.163265,277.521647,237,407,472,672,1359
Load time,49,6708.040816,7900.560014,577,2543,3759,7234,43065
Speed Index,49,3429.714286,2550.866487,600,1525,2675,4773,10769
Start render,49,1918.367347,1577.666192,600,1100,1500,2100,10700


### Step2: Normalize each parameter. After normalization, each parameter will have a value in between 0 and 1. 

If you want to normalize you data you can do as you suggest and simply calculate:
                 $$ zi=xi−min(x)/max(x)−min(x) $$
where x=(x1,...,xn) and zi is now your ith normalized data. As a proof of concept (although you did not ask for it) here is some R code and accompanying graph to illustrate this point:

In [70]:
def normalize(x, param, df_param):
    min_val = df_param.loc[param][0]
    max_val = df_param.loc[param][2]
    
    return (x - min_val)/(max_val - min_val)
    

In [71]:
normalize(700, 'Speed Index', df_param)

0.0098338086340839809

######  Step 3: iterate through each parameter and assign it points (from 1 to 100) on the basis of its normalized scale. For example, after normalization speed index is 0.2 for one website. Thinking intuitively, speed value should be more for good ranking and good performance. Now, we design our scale from 0 to 1 in such a way that high speed = high points. Example 1: calculating speed index

In [76]:
from sklearn import preprocessing

x = df_set.drop(['Site name'], axis=1)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled, columns= ['(Doc complete) Byets in', '(Doc complete) Requests','(Fully loaded) Bytes in','(Fully loaded) Requests',
          '(Fully loaded) Time','DOM elements','First byte','Load time','Speed Index','Start render',])
df=df.set_index(df_set['Site name'])
df.to_csv('./bokeh_app/data/normized.csv')

--------------------------------------------------------------------------------------------------------------------------------
$$  If (speed > =0 and speed <= .1) {  $$  
$$  //assign lowest points to the speed parameter variable. Eg = assign 10 points $$ 
$$  If(speed >= 0.2 and speed =< .3){ $$ 
$$  //Assign 20 points $$ 
$$  If(speed > = .3 and speed =<.4 $$ 
$$  //Assign 30 points $$ 
$$  And so on – for speed index - the points we assign increases as the value of the parameter lies in $$ 
$$  the range closer to 1. $$ 

Example 2: calculating load time
If (load time > =0 and speed <= .1) {

//assign highest points to the speed parameter variable. Eg = assign 90 points
If(speed >= 0.1 and speed =< .2){
// Assign 80 points
If(speed > = .2 and speed =<.3)
//Assign 70 points
And so on .. for load time - the points decreases as the value of the parameter lying in the range
is closer to 0.
------------------------------------------------------------------------------------------------------------------------------------------

Likewise, we do this step for each parameter for each website.
At the end of this step, we get total optimal points for each website having: speed points + load
time points + ...all other parameter points.

Step 4:
Now, we have the total points depending on the scale of each of the 100 websites calculated.

Step 5:
Compare the optimal points calculated now to get the ranking. As we have normalized the scale,
and have the model having total optimal points for each website.

Step 6:
Sort the total points of each website and get the ranking info depending on the sort. One with
the maximum points calculated is the ranked first and so on..

Step 7 : Once we have the ranking, if user wants to improve the ranking of a given website, so
basically increase the points for that website.

//Calculate increase of points required to make a website No.1
Points_rank1_website = p1;
Current_ Points_of_website_user_wants_to_improve = p2;
Points_needed = (points of rank 1 website – point of website user wants to improve to rank 1)
//Then increase those many points

Points_of_website_user_wants_to_improve = Points_of_website_user_wants_to_improve +
points_needed;

Step 7:
Depending on the new points, again sort the list and display ranking. Now user has the provision
to improve the ranking of any website and display it.